# Optimization Project

## 1. Introduction

Professor Karle Flanagan tasked us to build a model that will optimize her STAT 107 course staff schedule for the Spring 2025 semester that she anticipates will have 41 lab sessions, 21 teaching assistants and 50 course assistants. For this project, she gave us her current Fall 2024 semester scheduling system, so our model will be based off of this data.

Karle's current process is inefficient and tedious with numerous spreadsheets and a preference score system that she estimates takes 8 to 10 hours to complete. She would like a faster way to create her schedule and make sure students like the shifts they are assigned to.

Our model formulation is based off of classic scheduling problems that we learned in STAT 430. It requires information about the availability of each person, and in Karle's case, the preference scores as well. These parameters can easily be modeled as binary variables, which means yes/no response. For example: Is the CA available = yes/no, prefers this shift = yes/no?

We were able to create a scheduling model that meets Karle's main goals and create spreadsheets for assigned shifts by shift and assigned shifts by individual similar to the data Karle gave us. The model also gives some useful metrics such as the percentage of preferences met.

## 2. Problem

The main goal is to maximize the student teaching assistant (TA) and course assistant (CA) availability preferences for lab and office hour shifts and meet at least 90% of their preferences. The client also wished to meet the TA contract hour requirements of each TA working 2 to 3 lab shifts and exactly 1 office hour shift, and the guarantee of CAs working 4-6 hours/week to be the most important.

The optimization problem in our model required some stipulations due to the nature of the data. We are unable to ensure that some days are simply more popular than others for the course staff to mark as available, so we did not include Karle's request that more staff be assigned on busy days. Though Karle did not give this soft constraint, we had to set upper limits on number of each type of staff for office hour shifts to prevent one shift from having over 20 staff working in it. This is a flexible constraint that can be adjusted depending on Karle's needs.

Another stipulation is that we ignored the notes students gave on the spreadsheet Karle gave us because students sometimes voiced conflicting information on what they wanted and what they filled out on the schedule sheet.

According to our client, Karle, she found these goals and stipulations to be the most important: nobody is assigned to a shift they are not available for and that each shift meets the minimum staffing requirements. She considered the maximum caps of number of CAs and TAs assigned to office hours and labs to be flexible.

## 3. Model


<h3><u>Input Parameters and Sets:</h3></u>

*Sets*

$S$: A set of all staff members available for scheduling.
* $S_{TA}$ ⊆ $S$: Staff members assigned as TAS.
* $S_{CA}$ ⊆ $S$: Staff members assigned as CAS.

$M$: Set of all shifts to be scheduled.
* $M_{OH}$ ⊆ $M$: Shifts labeled as Office Hours (OH).
* $M_{LA}$ ⊆ $M$: Shifts labeled as lab shifts (LA).
<br><br>

*Input Parameters*

$Preference_{s,m} =
\left\{
\begin{array}{ll}
1 & \text{if } s \text{ prefers shift } m, \\
0 & \text{otherwise.}
\end{array}
\right.$
<br>

$Availability_{s,m} =
\left\{
\begin{array}{ll}
1 & \text{if } s \text{ is available for shift } m, \\
0 & \text{otherwise.}
\end{array}
\right.$
<br>

$Overlap_{m_i,m_j} =
\left\{
\begin{array}{ll}
\text{True} & \text{if shifts } m_i \text{ and } m_j \text{ overlap,} \\
\text{False} & \text{otherwise.}
\end{array}
\right.$
<br><br>

<h3><u>Decision Variables:</h3></u>

$x_{s,m} =
\left\{
\begin{array}{ll}
1 & \text{if staff member } s \text{ is assigned shift } m, \\
0 & \text{otherwise.}
\end{array}
\right.$
<br><br>

<h3><u>Objective Function:</h3></u>

$Maximize \sum\limits_{s \in S} \sum\limits_{m \in M} Preference_{s,m} \cdot x_{s,m}$
<br><br>
We aim to maximize total preferences by assigning staff members to shifts based on individual preferences. This is achieved by summing the preference scores for each shift assignment
<br><br>

<h3><u>Constraints</h3></u>

1.  **Each office hour must have at least 1 TA and 4 CAs. The maximum number of TAs and CAs each OH can have is 3 and 7, respectively:**
  
  * $1 \leq \sum\limits_{s \in S_{TA} } x_{s,m} \leq 3, \quad \forall m \in M_{OH}$

  * $4 \leq \sum\limits_{s \in S_{CA} } x_{s,m} \leq 7, \quad \forall m \in M_{OH}$


2.  **Each lab shift must have exactly 1 TA and 2 CAs:**

   * $\sum\limits_{s \in S_{TA} } x_{s,m} = 1, \quad \forall m \in M_{LA}$

   * $\sum\limits_{s \in S_{CA} } x_{s,m} = 2, \quad \forall m \in M_{LA}$

3. **Each TA can only work a minimum of 2 and a maximum of 3 lab shifts, and exactly 1 office hour shift:**

  * $2 \leq \sum\limits_{m \in M_{LA} } x_{s,m} \leq 3, \quad \forall s \in S_{TA}$

  * $\sum\limits_{m \in M_{OH} } x_{s,m} = 1, \quad \forall s \in S_{TA}$

4. **Each CA must work at least 1 lab and can work 0 or more OH. The maximum labs a CA can work is 3, and the maximum OH a CA can work is 2:**

  * $1 \leq\sum\limits_{m \in M_{LA} } x_{s,m} \leq 3, \quad \forall s \in S_{CA}$
  
  * $0 \leq\sum\limits_{m \in M_{OH} } x_{s,m} \leq 2, \quad \forall s \in S_{CA}$

5. **Staff members can only work shifts they are available for:**

  * $x_{s,m} = 0 \quad \text{if } Availability_{s,m} = 0, \quad \forall s \in S, \forall m \in M$

6. **No overlapping shifts are allowed for the same TA or CA:**

  * $x_{s,m_i} + x_{s,m_j} \leq 1 \quad \text{if } \text{Overlap}_{m_i, m_j} = \text{True}, \quad \forall s \in S, \forall(m_i, m_j) \in M$


In [ ]:
import pandas as pd
import pulp

In [ ]:
df=pd.read_csv('STAT107_shift_availability.csv')
df.head()

,Staff Member,Role,M_OH_13-15,M_OH_15-17,W_OH_13-15,W_OH_15-17,TH_OH_15-17,F_OH_15-17,W_LA_13-13:50_Y17,W_LA_13-13:50_Y16,...,TH_LA_15-15:50_Y36,TH_LA_16-16:50_Y20,TH_LA_16-16:50_Y14,TH_LA_17-17:50_Y15,F_LA_13-13:50_Y32,F_LA_13-13:50_Y30,F_LA_13-13:50_Y25,F_LA_14-14:50_Y33,F_LA_14-14:50_Y31,F_LA_14-14:50_Y26
0,Evangelos,TA,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Jake Hunnius,TA,1,1,1,1,0,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,Tyler Hecht,TA,0,1,0,1,0,1,1,1,...,0,1,1,1,1,1,1,0,0,0
3,Mallory Klostermann,TA,0,1,0,1,1,0,0,0,...,1,1,1,1,0,0,0,0,0,0
4,Eric Wayman,TA,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,0,0,0


In [ ]:
df_pref = pd.read_csv('STAT107_shift_pref.csv')
df_pref.head()

,Staff Member,Role,M_OH_13-15,M_OH_15-17,W_OH_13-15,W_OH_15-17,TH_OH_15-17,F_OH_15-17,W_LA_13-13:50_Y17,W_LA_13-13:50_Y16,...,TH_LA_15-15:50_Y36,TH_LA_16-16:50_Y20,TH_LA_16-16:50_Y14,TH_LA_17-17:50_Y15,F_LA_13-13:50_Y32,F_LA_13-13:50_Y30,F_LA_13-13:50_Y25,F_LA_14-14:50_Y33,F_LA_14-14:50_Y31,F_LA_14-14:50_Y26
0,Evangelos,TA,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Jake Hunnius,TA,0,0,1,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Tyler Hecht,TA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mallory Klostermann,TA,0,0,0,1,1,0,0,0,...,1,1,1,0,0,0,0,0,0,0
4,Eric Wayman,TA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
staff_members = df['Staff Member'].tolist()
roles = dict(zip(df['Staff Member'], df['Role ']))
shift_columns = df.columns[2:].tolist()

In [ ]:
staff_members[0]

'Evangelos'

In [ ]:
next(iter(roles.items()))

('Evangelos', 'TA')

In [ ]:
shift_columns[0]

'M_OH_13-15'

#### <u> Input Parameters and Sets</u>

**1. Below is the dictionary that holds the shift availability dataset:**
* Data cleaning: From the original dataset, :( - replaced with 0 (staff is not available at that time), 2-6 - replaced with 1 (staff is available at that time).

In [ ]:
#dictionary hold shift availability data
availability = {}
for _, row in df.iterrows():
    for shift in shift_columns:
        availability[(row['Staff Member'], shift)] = row[shift]

In [ ]:
next(iter(availability.items()))

(('Evangelos', 'M_OH_13-15'), 1)

Evangelos (TA), marked this office hour shift as available (can be assigned to it).

We get list of staff member names, a dictionary of their name and role (TA or CA) and a list of unique shifts.

**2. Below is a dictionary that holds the shift preference dataset:**
* Data cleaning: From the original dataset, :(, 2-6 - replaced with 0 (staff do not prefer these times), 1 is kept the same (indicating that this time is preferred).

In [ ]:
#dictionary of preference
preference = {}
for _, row in df_pref.iterrows():
    for shift in shift_columns:
        preference[(row['Staff Member'], shift)] = row[shift]

In [ ]:
next(iter(preference.items()))

(('Evangelos', 'M_OH_13-15'), 1)

Evangelos (TA), marked this office hour shift as preferred.

**3. Working with Overlapping shifts:**  In order to work with the non-overlaping constraints, it is important to parse the shift time (split times into day, start-time, and end-time) and check for shift overlap. For example, someone assigned an OH/Lab at 13:00 cannot work another shift at the same time. (they cannot be at two places at once). The code below parses the shift time and returns either true or false depending on whether or not two shifts (shift1 and shift2) overlaps at the same time and on the same day. This shifts_overlap function is used later for constraint 6 in the model which checks to make sure the same staff is not assigned different shifts on the same day and time.

In [ ]:
def parse_shift_time(shift_name):
    parts = shift_name.split('_')
    day = parts[0]
    time_block = parts[2]
    start_str, end_str = time_block.split('-')
    def to_decimal(t_str):
        if ':' in t_str:
            hh, mm = t_str.split(':')
            return int(hh) + int(mm)/60.0
        else:
            return float(t_str)
    start_time = to_decimal(start_str)
    end_time = to_decimal(end_str)
    return day, start_time, end_time

In [ ]:
#dictionary to store day/time info for each shift
shift_times = {}
for shift in shift_columns:
    day, start_time, end_time = parse_shift_time(shift)
    shift_times[shift] = (day, start_time, end_time)

def shifts_overlap(shift1, shift2):
    d1, s1, e1 = shift_times[shift1]
    d2, s2, e2 = shift_times[shift2]
    if d1 != d2:
        return False
    return s1 < e2 and s2 < e1


#### <u>Decision Variables</u>
1 if the staff member is assigned to the shift, 0 otherwise

In [ ]:
shift_problem = pulp.LpProblem("Shift_Scheduling", pulp.LpMaximize)

### Decision Variables
# Shift Availability: 1 if the staff member is assigned to the shift, 0 otherwise
assignments = pulp.LpVariable.dicts("Shift_Assignment",
                                   [(staff, shift) for staff in staff_members for shift in shift_columns],
                                   cat='Binary')

#### <u>Model</u>

In [ ]:

### Objective Function ------- Maximize Preference
shift_problem += pulp.lpSum(preference.get((staff, shift), 0) * assignments[staff, shift]
    for staff in staff_members
    for shift in shift_columns), "Maximize_Preferences"

### Constraints

# 1. Each Office Hours (OH) shift must have at least 1 TA and 4 CAs
oh_shifts = [shift for shift in shift_columns if 'OH' in shift]
for shift in oh_shifts:
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'TA']) >= 1, f"Min_TA_{shift}"
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'TA']) <= 3, f"Max_TA_{shift}"
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'CA']) >= 4, f"Min_CA_{shift}"
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'CA']) <= 7, f"Max_CA_{shift}"


# 2. Each Lab shift must have exactly 1 TA and 2 CAs
lab_shifts = [shift for shift in shift_columns if 'LA' in shift]
for shift in lab_shifts:
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'TA']) == 1, f"TA_Lab_{shift}"
    shift_problem += pulp.lpSum([assignments[staff, shift] for staff in staff_members if roles.get(staff) == 'CA']) == 2, f"CA_Lab_{shift}"

# 3. Each TA can only work 2 to 3 lab shifts and exactly 1 office hour shift (based on Karle's sheets)
for staff in staff_members:
    if roles.get(staff) == 'TA':
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in lab_shifts]) >= 2, f"Min_Labs_TA_{staff}"
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in lab_shifts]) <= 3, f"Max_Labs_TA_{staff}"
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in oh_shifts]) == 1, f"Min_OH_TA_{staff}"

# # 4. Each CA must work atleast 1 labs and can work 0 or more OH (to keep it balanced say max is 2)
for staff in staff_members:
    if roles.get(staff) == 'CA':
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in lab_shifts]) >= 1, f"Min_Labs_CA_{staff}"
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in lab_shifts]) <= 3, f"Max_Labs_CA_{staff}"
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in oh_shifts]) >= 0, f"Min_OH_CA_{staff}"
        shift_problem += pulp.lpSum([assignments[staff, shift] for shift in oh_shifts]) <= 2, f"Max_OH_CA_{staff}"


# 5. Availability: staff member only works shift if they mark it as available
for staff in staff_members:
    for shift in shift_columns:
        # don't assign shift to staff if not available
        if availability.get((staff, shift), 0) == 0:
            shift_problem += assignments[staff, shift] == 0, f"Availability_{staff}_{shift}"

#6. OVERLAP CONSTRAINT (This is to make sure that no timings for the same TA/CA overlap for both OH and Labs )
for staff in staff_members:
    for i, shift1 in enumerate(shift_columns):
        for shift2 in shift_columns[i+1:]:
            if shifts_overlap(shift1, shift2):
                #Staff cannot do both shift1 and shift2
                shift_problem += assignments[staff, shift1] + assignments[staff, shift2] <= 1, f"No_Overlap_{staff}_{shift1}_{shift2}"

shift_problem.solve()


1

In [ ]:
print("Solver Status:", pulp.LpStatus[shift_problem.status])


Solver Status: Optimal


Below are the final schedules.

### Model: Schedule Assignment by Shift

In [ ]:
shift_assignment_data = []

for shift in shift_columns:
    assigned_tas = sum([pulp.value(assignments[staff, shift]) for staff in staff_members if roles.get(staff) == 'TA'])
    assigned_cas = sum([pulp.value(assignments[staff, shift]) for staff in staff_members if roles.get(staff) == 'CA'])

    # Add the shift data to the list
    shift_assignment_data.append({
        "Shift": shift,
        "Number of TAs Assigned": assigned_tas,
        "Number of CAs Assigned": assigned_cas
    })

df_schedule_by_shift  = pd.DataFrame(shift_assignment_data)
# df_schedule_by_shift.to_csv("schedule_by_shift.csv")
df_schedule_by_shift

,Shift,Number of TAs Assigned,Number of CAs Assigned
0,M_OH_13-15,3.0,7.0
1,M_OH_15-17,3.0,7.0
2,W_OH_13-15,1.0,6.0
3,W_OH_15-17,3.0,7.0
4,TH_OH_15-17,2.0,4.0
5,F_OH_15-17,3.0,7.0
6,W_LA_13-13:50_Y17,1.0,2.0
7,W_LA_13-13:50_Y16,1.0,2.0
8,W_LA_13-13:50_Y02,1.0,2.0
9,W_LA_13-13:50_Y01,1.0,2.0


### Model: Schedule Assignment by Person

In [ ]:
# Create Dataframe of Staff Assignments from Pulp Model
staff_data = []

for staff in staff_members:
    # Dataframe Structure
    staff_info = {
        "Name": staff,
        "Role": roles.get(staff),
        "OH Shifts": [],
        "Lab Shifts": [],
        "Num OH Shifts": 0,
        "Num Lab Shifts": 0
    }

    # Loop through each shift
    for shift in shift_columns:
        if pulp.value(assignments[staff, shift]) == 1:  # If the staff is assigned to this shift
            availability_status = availability.get((staff, shift), 0)  # Get availability status (0 if not available)

            # Print out if assigned to a shift they were not available
            if availability_status == 0:
                print(f"{staff} assigned to {shift} when marked availability was {availability_status}. Please update")

            # Add to the correct OH or Lab shift space
            if 'OH' in shift:
                staff_info["OH Shifts"].append((shift, availability_status))
                staff_info["Num OH Shifts"] += 1  # Increment the OH shift count
            elif 'LA' in shift:
                staff_info["Lab Shifts"].append((shift, availability_status))
                staff_info["Num Lab Shifts"] += 1  # Increment the Lab shift count
    # Append to list
    staff_data.append(staff_info)

# Convert the list of staff data into the DataFrame
df_schedule_by_person = pd.DataFrame(staff_data)
# df_schedule_by_person.to_csv("schedule_by_person.csv")
df_schedule_by_person

,Name,Role,OH Shifts,Lab Shifts,Num OH Shifts,Num Lab Shifts
0,Evangelos,TA,"[(W_OH_15-17, 1)]","[(W_LA_13-13:50_Y17, 1), (W_LA_14-14:50_Y04, 1...",1,3
1,Jake Hunnius,TA,"[(W_OH_15-17, 1)]","[(W_LA_13-13:50_Y02, 1), (W_LA_14-14:50_Y19, 1)]",1,2
2,Tyler Hecht,TA,"[(F_OH_15-17, 1)]","[(TH_LA_16-16:50_Y20, 1), (F_LA_13-13:50_Y30, 1)]",1,2
3,Mallory Klostermann,TA,"[(TH_OH_15-17, 1)]","[(W_LA_15-15:50_Y24, 1), (W_LA_16-16:50_Y22, 1...",1,3
4,Eric Wayman,TA,"[(W_OH_13-15, 1)]","[(W_LA_17-17:50_Y29, 1), (TH_LA_10-10:50_Y08, 1)]",1,2
5,Nathaniel Butler,TA,"[(M_OH_15-17, 1)]","[(W_LA_15-15:50_Y23, 1), (W_LA_16-16:50_Y06, 1...",1,3
6,Omkar,TA,"[(F_OH_15-17, 1)]","[(W_LA_14-14:50_Y03, 1), (TH_LA_11-11:50_Y10, 1)]",1,2
7,Mukhil,TA,"[(M_OH_13-15, 1)]","[(W_LA_16-16:50_Y28, 1), (TH_LA_15-15:50_Y36, 1)]",1,2
8,Shuning,TA,"[(M_OH_13-15, 1)]","[(TH_LA_12-12:50_Y34, 1), (TH_LA_13-13:50_Y12,...",1,2
9,Mitchell,TA,"[(TH_OH_15-17, 1)]","[(TH_LA_17-17:50_Y15, 1), (F_LA_13-13:50_Y25, 1)]",1,2


## 4. Numerical Analysis

### **Preference Percentage Met**
After using this semester's data in our model, we have evaluated out of all shifts assigned, how many of those shifts were classified as "preferred" shifts. As shown below, we hit a percentage of 92.55% of overall preferred shifts met.

This model ensures that 100% of assigned shifts align with staff availability and meeting all other constraints, while maximizing preference and acheiving **92.55%** of shift preferences in our optimal solution.

Preferences in our model were based on shifts only scored as a 1, the staff's top choice. If we change our metrics and include scores just as '2', we could further **increase** this percentage and total preferred shifts met.

In [ ]:
#calculate total shifts assigned
total_shifts_assigned = sum(
    assignments[staff, shift].varValue
    for staff in staff_members
    for shift in shift_columns
)

#calculate total preferred shifts assigned
total_pref_shifts_assigned = pulp.value(shift_problem.objective)

#overall percentage of preferred shifts met
percent_pref_shifts_met = (total_pref_shifts_assigned / total_shifts_assigned) * 100

print(f"Total Shifts Assigned: {total_shifts_assigned}")
print(f"Total Preferred Shifts Assigned: {total_pref_shifts_assigned}")
print(f"Overall Percentage of Preferred Shifts Met: {percent_pref_shifts_met:.2f}%")

Total Shifts Assigned: 161.0
Total Preferred Shifts Assigned: 149.0
Overall Percentage of Preferred Shifts Met: 92.55%


To get the final output **(OPTIMAL SOLUTION)**, run the code below to get the schedule assignment by person. The solution will be outputted as a csv file which our client may use as they wish :

In [ ]:
df_schedule_by_person.to_csv("schedule_assignment_by_person.csv")

* For further information, to check out the number of TA's/CA's assigned to each shift, run the code below:

In [ ]:
df_schedule_by_shift.to_csv("schedule_by_shift.csv")

### **Model Solution Time**

Current method time: ~8-10 hours manually

Optimization Model time: under 1 second run time

After inputting the correctly formatted csv files for the semester into this model, the process should only take a few minutes, *significantly* more efficent than the original method. Our model allows for changes to be made in the staff's availability and preferences, and will still run the solution quickly, while meeting all constraints.

In [ ]:
print(f"Solver Time: {shift_problem.solutionTime} seconds")

Solver Time: 0.4539220333099365 seconds


In [ ]:
current_seconds = 10*60*60
current_seconds/0.453922

79308.77992254175

Our current solution is about **79,308** times faster than the manual method!

### **Metrics of Success**

**Enhanced Organization**
- Outputs clear schedules:
    - By each shift:
        - Number of TAs/CAs working
        
    - By each person:
       - Staff members listed with assigned shifts
       - Separation of OH and Lab shifts
       - Roles clearly identified
       - Number of OHs/Labs assigned per person

**Preference Satisfaction**
- Karle aimed for total preferences met to reach 90%
    - The current model *exceeds* 90%, while only accounting for top-choice shifts, scored 1
   
- Time Efficiency
    - **10 hours** manually to **0.25 seconds** run time

**Adaptability and User Effort**
- Adapts easily to changes in availability and preferences
- Input correctly formatted availability/preference CSV files to align with model formation

**Error Reduction**
- Model forces all constraints to be met for a feasible output, otherwise will not return a solution
- Eliminates manual errors created in original process

**Client Benefit**
- Time saved
- Staff satisfaction
- Operational clarity
    - easily understood outputs, requires little work in coding to adjust inputted data
- Future-proofing
    - Scalable and adaptable solutions for any evolving needs, can adjust limits on hours, shifts, etc.

## 5. Conclusion

The STAT 107 scheduling optimization project demonstrates the power of leveraging mathematical modeling and computational tools to address complex scheduling challenges. Our solution achieved a 92.55% preference satisfaction rate, successfully assigning 161 shifts while ensuring that all constraints, including availability, staffing requirements, and non-overlapping schedules, were met.

### Key Achievements

1. **Efficiency Gains:**  
   Our model reduced scheduling time from 10+ hours of manual effort to 0.24 seconds, representing a 150,000x increase in efficiency.

2. **Accuracy and Fairness:**  
   By automating the process, we eliminated human error and maximized preference satisfaction, ensuring staff members are assigned shifts they are both qualified for and available to work.

3. **Scalability and Adaptability:**  
   The model is highly adaptable, capable of accommodating dynamic input data and evolving requirements for future semesters.


### Impact

This optimized scheduling process not only meets the operational needs of STAT 107 but also significantly reduces administrative burden, allowing for more time to focus on teaching and mentoring. Furthermore, this approach is scalable to other departments and courses with similar scheduling needs, providing a framework for broader institutional improvements.

### Future Directions

To further enhance the model:  
- Incorporate additional factors like staff preferences beyond binary availability, proximity of shifts, and varying demand during peak hours.  
- Utilize feedback from stakeholders to refine constraints and objectives for an even higher satisfaction rate.

Through this project, we have demonstrated that data-driven decision-making can lead to impactful solutions, paving the way for continued innovation in academic scheduling.



## References

Data given to us by Karle and insight from the interview/followup interview